# 리뷰분석 플라스크에 추가되면 추가하고 git 에 올리자 !

In [2]:
!mkdir -p jobflask/static
!mkdir -p jobflask/templates
!touch jobflask/jobflask.py
!touch jobflask/templates/index.html
!tree jobflask

jobflask
├── hello.py
├── jobflask.py
├── static
└── templates
    └── index.html

2 directories, 3 files


In [4]:
!mkdir -p jobflask/mongodb
!tree jobflask

jobflask
├── jobflask.py
├── mongodb
├── static
└── templates
    └── index.html

3 directories, 2 files


### mongodb모듈생성

In [11]:
%%writefile jobflask/mongodb_ini.py
import configparser
import pymongo
def mongodb():
    #.ini 파일을 통해 몽고db 로그인
    config = configparser.ConfigParser()
    config.read('login.ini')
    user = config.get('ju', 'ID')
    pwd = config.get('ju', 'PW')
    ip = config.get('ju', 'IP')    
    account = f'mongodb://{user}:{pwd}@{ip}'
    server = pymongo.MongoClient(account, 27017) 
    db=server.jobplanet
    return db

Overwriting jobflask/mongodb_ini.py


### Flask 생성

In [10]:
%%writefile jobflask/jobflask.py
import pickle
import os
import pymongo
from flask import *
import configparser
from mongodb_ini import mongodb

app=Flask(__name__)


@app.route("/raw_review/<company_name>/<title2>")
def raw_review(company_name,title2):
    #jobplanet db의 review 테이블 접근
    db=mongodb()
    collection_re=db.review
    result= list(collection_re.find({'company_name':'{}'.format(company_name)}))[0]
    del result['_id']
    return jsonify(result)

#jobplanet raw_recruit_data
@app.route("/raw_recruit_jp/<company_name>/<title2>")
def raw_recruit_jp(company_name,title2):
    #jobplanet db의 recruit_jp 테이블 접근
    db=mongodb()
    collection_c_jp=db.recruit_jp
    result=list(collection_c_jp.find({"$and":[{"title2":'{}'.format(title2)},{"company_name":"{}".format(company_name)}]}))[0]
    del result['_id']
    return jsonify(result)

#wanted raw_recruit_data
@app.route("/raw_recruit_w/<company_name>/<title2>")
def raw_recruit_w(company_name,title2):
    #jobplanet db의 recruit_w테이블 접근
    db=mongodb()
    collection_c_w=db.recruit_w
    result=list(collection_c_w.find({"$and":[{"title2":'{}'.format(title2)},{"company_name":"{}".format(company_name)}]}))[0]
    del result['_id']
    return jsonify(result)

@app.route("/jobplanet/<company_name>/<title2>")
def jobplanet(company_name,title2):
    return render_template('index_jp.html',company_name=company_name,title2=title2)

@app.route("/wanted/<company_name>/<title2>")
def wanted(company_name,title2):
    return render_template('index_w.html',company_name=company_name,title2=title2)
    
app.run(debug=True)

Overwriting jobflask/jobflask.py


### index_jp.html : jobplanet html

In [6]:
%%writefile jobflask/templates/index_jp.html
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <title>{{company_name}}</title>
    <link href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" rel="stylesheet">

    <style> 
    .title{
        width:300px;
        height:300px;
        position:absolute;
        left:50%;
        top:50%;
        margin-left:-150px;
        margin-top:-150px;
        font-weight: bold;
        font-size: 32px;
    }
    
    .data{
        text-align:center;
        font-weight: bold;
        font-size: 22px;
    }
    
    .data_c_jp{
        text-align:center;
        font-weight: bold;
        font-size: 22px;
    }
    

    </style>
</head>
<body>
    <div class="title" style="text-align:center;">
    <p id="name_title" >{{company_name}}</p>
    <p></p>
    <button type="button" id="getdescription_jp" class="btn btn-secondary" style="text-align:center;">Description Click</button>    
    <button type="button" id="getreview" class="btn btn-secondary" style="text-align:center;">Review Click</button>
    
    </div>
    
    <div class="data"></div>
    <div class="data_c_jp"></div>
    
    
    <div id="accordion" style="display:none;">
    <div class="card" >
        <div class="card-header" id="headingOne">
          <h5 class="mb-0">
            <button class="btn btn-link" data-toggle="collapse" data-target="#collapseOne" aria-expanded="true" aria-controls="collapseOne">
              Reveiw Link
            </button>
          </h5>
        </div>
        <div id="collapseOne" class="collapse show" aria-labelledby="headingOne" data-parent="#accordion">
          <div class="card-body">
          </div>
        </div>
      </div>
    
      <div class="card">
        <div class="card-header" id="headingTwo">
          <h5 class="mb-0">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseTwo" aria-expanded="true" aria-controls="collapseTwo">
              Review Num
            </button>
          </h5>
        </div>
        <div id="collapseTwo" class="collapse" aria-labelledby="headingTwo" data-parent="#accordion">
          <div class="card-body">
          </div>
        </div>
      </div>
    
      <div class="card">
        <div class="card-header" id="headingThree">
          <h5 class="mb-0">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseThree" aria-expanded="true" aria-controls="collapseThree">
            Review Stats
            </button>
          </h5>
        </div>
        <div id="collapseThree" class="collapse" aria-labelledby="headingThree" data-parent="#accordion">
          <div class="card-body">
          </div>
        </div>
      </div>
    
     <div class="card">
        <div class="card-header" id="headingFour">
          <h5 class="mb-0">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseFour" aria-expanded="true" aria-controls="collapseFour">
              Person
            </button>
          </h5>
        </div>
        <div id="collapseFour" class="collapse" aria-labelledby="headingFour" data-parent="#accordion">
          <div class="card-body">
          </div>
        </div>
      </div>
    
     <div class="card">
        <div class="card-header" id="headingFive">
          <h5 class="mb-0">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseFive" aria-expanded="true" aria-controls="collapseFive">
              Title
            </button>
          </h5>
        </div>
        <div id="collapseFive" class="collapse" aria-labelledby="headingFive" data-parent="#accordion">
          <div class="card-body">
          </div>
        </div>
      </div>
    
      <div class="card">
        <div class="card-header" id="headingSix">
          <h5 class="mb-0">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseSix" aria-expanded="true" aria-controls="collapseSix">
              Strength
            </button>
          </h5>
        </div>
        <div id="collapseSix" class="collapse" aria-labelledby="headingSix" data-parent="#accordion">
          <div class="card-body">
          </div>
        </div>
      </div>
    
      <div class="card">
        <div class="card-header" id="headingSeven">
          <h5 class="mb-0">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseSeven" aria-expanded="true" aria-controls="collapseSeven">
            Weakness
            </button>
          </h5>
        </div>
        <div id="collapseSeven" class="collapse" aria-labelledby="headingSeven" data-parent="#accordion">
          <div class="card-body">
          </div>
        </div>
      </div>
    
     <div class="card">
        <div class="card-header" id="headingEight">
          <h5 class="mb-0">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseEight" aria-expanded="true" aria-controls="collapseEight">
              Want
            </button>
          </h5>
        </div>
        <div id="collapseEight" class="collapse" aria-labelledby="headingEight" data-parent="#accordion">
          <div class="card-body">
          </div>
        </div>
      </div>

    </div>

    <div id="accordion_c_jp" style="display:none;">
    <div class="card" >
        <div class="card-header" id="headingOne_c_jp">
          <h5 class="mb-0">
            <button class="btn btn-link" data-toggle="collapse" data-target="#collapseOne_c_jp" aria-expanded="true" aria-controls="collapseOne_c_jp">
              Homepage
            </button>
          </h5>
        </div>
        <div id="collapseOne_c_jp" class="collapse" aria-labelledby="headingOne_c_jp" data-parent="#accordion_c_jp">
          <div class="card-body">
          </div>
        </div>
      </div>
    
      <div class="card">
        <div class="card-header" id="headingTwo_c_jp">
          <h5 class="mb-0">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseTwo_c_jp" aria-expanded="true" aria-controls="collapseTwo_c_jp">
              Title
            </button>
          </h5>
        </div>
        <div id="collapseTwo_c_jp" class="collapse" aria-labelledby="headingTwo_c_jp" data-parent="#accordion_c_jp">
          <div class="card-body">
          </div>
        </div>
      </div>
    
      <div class="card">
        <div class="card-header" id="headingThree_c_jp">
          <h5 class="mb-0">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseThree_c_jp" aria-expanded="true" aria-controls="collapseThree_c_jp">
            Introduction
            </button>
          </h5>
        </div>
        <div id="collapseThree_c_jp" class="collapse" aria-labelledby="headingThree_c_jp" data-parent="#accordion_c_jp">
          <div class="card-body">
          </div>
        </div>
      </div>
    
     <div class="card">
        <div class="card-header" id="headingFour_c_jp">
          <h5 class="mb-0">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseFour_c_jp" aria-expanded="true" aria-controls="collapseFour_c_jp">
              Task 
            </button>
          </h5>
        </div>
        <div id="collapseFour_c_jp" class="collapse" aria-labelledby="headingFour_c_jp" data-parent="#accordion_c_jp">
          <div class="card-body">
          </div>
        </div>
      </div>
    
     <div class="card">
        <div class="card-header" id="headingFive_c_jp">
          <h5 class="mb-0">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseFive_c_jp" aria-expanded="true" aria-controls="collapseFive_c_jp">
              Ability
            </button>
          </h5>
        </div>
        <div id="collapseFive_c_jp" class="collapse" aria-labelledby="headingFive_c_jp" data-parent="#accordion_c_jp">
          <div class="card-body">
          </div>
        </div>
      </div>
    
      <div class="card">
        <div class="card-header" id="headingSix_c_jp">
          <h5 class="mb-0">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseSix_c_jp" aria-expanded="true" aria-controls="collapseSix_c_jp">
              Capability
            </button>
          </h5>
        </div>
        <div id="collapseSix_c_jp" class="collapse" aria-labelledby="headingSix_c_jp" data-parent="#accordion_c_jp">
          <div class="card-body">
          </div>
        </div>
      </div>
    
      <div class="card">
        <div class="card-header" id="headingSeven_c_jp">
          <h5 class="mb-0">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseSeven_c_jp" aria-expanded="true" aria-controls="collapseSeven_c_jp">
            Industry
            </button>
          </h5>
        </div>
        <div id="collapseSeven_c_jp" class="collapse" aria-labelledby="headingSeven_c_jp" data-parent="#accordion_c_jp">
          <div class="card-body">
          </div>
        </div>
      </div>
    
     <div class="card">
        <div class="card-header" id="headingEight_c_jp">
          <h5 class="mb-0">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseEight_c_jp" aria-expanded="true" aria-controls="collapseEight_c_jp">
              Duration
            </button>
          </h5>
        </div>
        <div id="collapseEight_c_jp" class="collapse" aria-labelledby="headingEight_c_jp" data-parent="#accordion_c_jp">
          <div class="card-body">
          </div>
        </div>
      </div>
    
     <div class="card">
        <div class="card-header" id="headingNine_c_jp">
          <h5 class="mb-0">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseNine_c_jp" aria-expanded="true" aria-controls="collapseNine_c_jp">
              Registration
            </button>
          </h5>
        </div>
        <div id="collapseNine_c_jp" class="collapse" aria-labelledby="headingNine_c_jp" data-parent="#accordion_c_jp">
          <div class="card-body">
          </div>
        </div>
      </div>
    
     <div class="card">
        <div class="card-header" id="headingTen_c_jp">
          <h5 class="mb-0">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseTen_c_jp" aria-expanded="true" aria-controls="collapseTen_c_jp">
              Treatment
            </button>
          </h5>
        </div>
        <div id="collapseTen_c_jp" class="collapse" aria-labelledby="headingTen_c_jp" data-parent="#accordion_c_jp">
          <div class="card-body">
          </div>
        </div>
      </div>
    
     <div class="card">
        <div class="card-header" id="headingEleven_c_jp">
          <h5 class="mb-0">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseEleven_c_jp" aria-expanded="true" aria-controls="collapseEleven_c_jp">
              Process
            </button>
          </h5>
        </div>
        <div id="collapseEleven_c_jp" class="collapse" aria-labelledby="headingEleven_c_jp" data-parent="#accordion_c_jp">
          <div class="card-body">
          </div>
        </div>
      </div>
    
     <div class="card">
        <div class="card-header" id="headingTwelve_c_jp">
          <h5 class="mb-0">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseTwelve_c_jp" aria-expanded="true" aria-controls="collapseTwelve_c_jp">
              Etc
            </button>
          </h5>
        </div>
        <div id="collapseTwelve_c_jp" class="collapse" aria-labelledby="headingTwelve_c_jp" data-parent="#accordion_c_jp">
          <div class="card-body">
          </div>
        </div>
      </div>
    
     <div class="card">
        <div class="card-header" id="headingThirteen_c_jp">
          <h5 class="mb-0">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseThirteen_c_jp" aria-expanded="true" aria-controls="collapseThirteen_c_jp">
              Welfare
            </button>
          </h5>
        </div>
        <div id="collapseThirteen_c_jp" class="collapse" aria-labelledby="headingThirteen_c_jp" data-parent="#accordion_c_jp">
          <div class="card-body">
          </div>
        </div>
      </div>
    </div>
    
   


    <script src="https://cdnjs.cloudflare.com/ajax/libs/jquery/3.4.1/jquery.js"></script>
    <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/js/bootstrap.min.js"></script>
    <script type="text/javascript">
        $(document).ready(function(){
                $('#getdescription_jp').on('click', function(){
                $('#getdescription_jp').hide();
                $('#name_title').hide();
                $('#accordion_c_jp').show()
                $.getJSON("/raw_recruit_jp/{{company_name}}/{{title2}}", function(data_c_jp){
                    console.log(data_c_jp);
                    var company_name="<p>"+data_c_jp.company_name+"</p>"
                        homepage = "<a href="+data_c_jp.homepage +">"+data_c_jp.homepage+"</a>";
                        title = "<p>" + data_c_jp.title + "</p>";
                        introduction = "<p>" + data_c_jp.introduction + "</p>";
                        task = "<p>" + data_c_jp.task+"  "+ + "</p>";
                        ability = "<p>" + data_c_jp.ability + "</p>";
                        capability = "<p>" + data_c_jp.capability + "</p>";
                        industry = "<p>" + data_c_jp.industry + "</p>";
                        duration = "<p>" + data_c_jp.duration + "</p>";
                        registration = "<p>" + data_c_jp.registration+"  "+ + "</p>";
                        treatment = "<p>" + data_c_jp.treatment + "</p>";
                        process = "<p>" + data_c_jp.process + "</p>";
                        etc = "<p>" + data_c_jp.etc + "</p>";
                        welfare = "<p>" + data_c_jp.welfare + "</p>";

                    $(".data_c_jp").append(company_name);
                    $("#collapseOne_c_jp").append(homepage);
                    $("#collapseTwo_c_jp").append(title);
                    $("#collapseThree_c_jp").append(introduction); 
                    $("#collapseFour_c_jp").append(task);
                    $("#collapseFive_c_jp").append(ability);
                    $("#collapseSix_c_jp").append(capability);
                    $("#collapseSeven_c_jp").append(industry);
                    $("#collapseEight_c_jp").append(duration);
                    $("#collapseNine_c_jp").append(registration);
                    $("#collapseTen_c_jp").append(treatment);
                    $("#collapseEleven_c_jp").append(process);
                    $("#collapseTwelve_c_jp").append(etc);
                    $("#collapseThirteen_c_jp").append(welfare);

                });
            });
            
            
                $('#getreview').on('click', function(){
                $('#getreview').hide();
                $('#name_title').hide();
                $('#accordion').show()
                $.getJSON("/raw_review/{{company_name}}/{{title2}}", function(data){
                    console.log(data);                   
                    var company_name="<p>"+data.company_name+"</p>"
                    var link = "<a href="+data.link +">"+data.link+"</a>";
                    var review_num = "<p>" + data.review_num + "</p>";
                    var stats = "<p>" + JSON.stringify(data.stats) + "</p>";
                        person = "<p>" + data.person+"  "+ + "</p>";
                        title = "<p>" + data.title + "</p>";
                        strength = "<p>" + data.strength + "</p>";
                        weakness = "<p>" + data.weakness + "</p>";
                        want = "<p>" + data.want + "</p>";
                        
                    $(".data").append(company_name);
                    $("#collapseOne").append(link);
                    $("#collapseTwo").append(review_num);
                    $("#collapseThree").append(stats); 
                    $("#collapseFour").append(person);
                    $("#collapseFive").append(title);
                    $("#collapseSix").append(strength);
                    $("#collapseSeven").append(weakness);
                    $("#collapseEight").append(want);
                    
                });
            });     
        });
        $('.collapse').collapse();
        
    </script>
</body>

</html>

Overwriting jobflask/templates/index_jp.html


### index_w.html : wanted html

In [7]:
%%writefile jobflask/templates/index_w.html
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <title>{{company_name}}</title>
    <link href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" rel="stylesheet">

    <style> 
    .title{
        width:300px;
        height:300px;
        position:absolute;
        left:50%;
        top:50%;
        margin-left:-150px;
        margin-top:-150px;
        font-weight: bold;
        font-size: 32px;
    }
    
    .data{
        text-align:center;
        font-weight: bold;
        font-size: 22px;
    }
    
    
     .data_c_w{
     text-align:center;
     font-weight: bold;
     font-size: 22px;
    }
    
    </style>
</head>
<body>
    <div class="title" style="text-align:center;">
    <p id="name_title" >{{company_name}}</p>
    <p></p>
    <button type="button" id="getdescription_w" class="btn btn-secondary" style="text-align:center;">Description Click</button>    
    <button type="button" id="getreview" class="btn btn-secondary" style="text-align:center;">Review Click</button>
    
    </div>
    
    <div class="data"></div>
    <div class="data_c_w"></div>
    
    
    <div id="accordion" style="display:none;">
    <div class="card" >
        <div class="card-header" id="headingOne">
          <h5 class="mb-0">
            <button class="btn btn-link" data-toggle="collapse" data-target="#collapseOne" aria-expanded="true" aria-controls="collapseOne">
              Reveiw Link
            </button>
          </h5>
        </div>
        <div id="collapseOne" class="collapse show" aria-labelledby="headingOne" data-parent="#accordion">
          <div class="card-body">
          </div>
        </div>
      </div>
    
      <div class="card">
        <div class="card-header" id="headingTwo">
          <h5 class="mb-0">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseTwo" aria-expanded="true" aria-controls="collapseTwo">
              Review Num
            </button>
          </h5>
        </div>
        <div id="collapseTwo" class="collapse" aria-labelledby="headingTwo" data-parent="#accordion">
          <div class="card-body">
          </div>
        </div>
      </div>
    
      <div class="card">
        <div class="card-header" id="headingThree">
          <h5 class="mb-0">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseThree" aria-expanded="true" aria-controls="collapseThree">
            Review Stats
            </button>
          </h5>
        </div>
        <div id="collapseThree" class="collapse" aria-labelledby="headingThree" data-parent="#accordion">
          <div class="card-body">
          </div>
        </div>
      </div>
    
     <div class="card">
        <div class="card-header" id="headingFour">
          <h5 class="mb-0">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseFour" aria-expanded="true" aria-controls="collapseFour">
              Person
            </button>
          </h5>
        </div>
        <div id="collapseFour" class="collapse" aria-labelledby="headingFour" data-parent="#accordion">
          <div class="card-body">
          </div>
        </div>
      </div>
    
     <div class="card">
        <div class="card-header" id="headingFive">
          <h5 class="mb-0">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseFive" aria-expanded="true" aria-controls="collapseFive">
              Title
            </button>
          </h5>
        </div>
        <div id="collapseFive" class="collapse" aria-labelledby="headingFive" data-parent="#accordion">
          <div class="card-body">
          </div>
        </div>
      </div>
    
      <div class="card">
        <div class="card-header" id="headingSix">
          <h5 class="mb-0">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseSix" aria-expanded="true" aria-controls="collapseSix">
              Strength
            </button>
          </h5>
        </div>
        <div id="collapseSix" class="collapse" aria-labelledby="headingSix" data-parent="#accordion">
          <div class="card-body">
          </div>
        </div>
      </div>
    
      <div class="card">
        <div class="card-header" id="headingSeven">
          <h5 class="mb-0">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseSeven" aria-expanded="true" aria-controls="collapseSeven">
            Weakness
            </button>
          </h5>
        </div>
        <div id="collapseSeven" class="collapse" aria-labelledby="headingSeven" data-parent="#accordion">
          <div class="card-body">
          </div>
        </div>
      </div>
    
     <div class="card">
        <div class="card-header" id="headingEight">
          <h5 class="mb-0">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseEight" aria-expanded="true" aria-controls="collapseEight">
              Want
            </button>
          </h5>
        </div>
        <div id="collapseEight" class="collapse" aria-labelledby="headingEight" data-parent="#accordion">
          <div class="card-body">
          </div>
        </div>
      </div>

    </div>

 
    
    <div id="accordion_c_w" style="display:none;">
    <div class="card" >
        <div class="card-header" id="headingOne_c_w">
          <h5 class="mb-0">
            <button class="btn btn-link" data-toggle="collapse" data-target="#collapseOne_c_w" aria-expanded="true" aria-controls="collapseOne_c_w">
              Title
            </button>
          </h5>
        </div>
        <div id="collapseOne_c_w" class="collapse" aria-labelledby="headingOne_c_w" data-parent="#accordion_c_w">
          <div class="card-body">
          </div>
        </div>
      </div>
    
      <div class="card">
        <div class="card-header" id="headingTwo_c_w">
          <h5 class="mb-0">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseTwo_c_w" aria-expanded="true" aria-controls="collapseTwo_c_w">
              Introduction
            </button>
          </h5>
        </div>
        <div id="collapseTwo_c_w" class="collapse" aria-labelledby="headingTwo_c_w" data-parent="#accordion_c_w">
          <div class="card-body">
          </div>
        </div>
      </div>
    
      <div class="card">
        <div class="card-header" id="headingThree_c_w">
          <h5 class="mb-0">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseThree_c_w" aria-expanded="true" aria-controls="collapseThree_c_w">
            Task
            </button>
          </h5>
        </div>
        <div id="collapseThree_c_w" class="collapse" aria-labelledby="headingThree_c_w" data-parent="#accordion_c_w">
          <div class="card-body">
          </div>
        </div>
      </div>
    
     <div class="card">
        <div class="card-header" id="headingFour_c_w">
          <h5 class="mb-0">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseFour_c_w" aria-expanded="true" aria-controls="collapseFour_c_w">
              Ability 
            </button>
          </h5>
        </div>
        <div id="collapseFour_c_w" class="collapse" aria-labelledby="headingFour_c_w" data-parent="#accordion_c_w">
          <div class="card-body">
          </div>
        </div>
      </div>
    
     <div class="card">
        <div class="card-header" id="headingFive_c_w">
          <h5 class="mb-0">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseFive_c_w" aria-expanded="true" aria-controls="collapseFive_c_w">
              Registration
            </button>
          </h5>
        </div>
        <div id="collapseFive_c_w" class="collapse" aria-labelledby="headingFive_c_w" data-parent="#accordion_c_w">
          <div class="card-body">
          </div>
        </div>
      </div>
    
      <div class="card">
        <div class="card-header" id="headingSix_c_w">
          <h5 class="mb-0">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseSix_c_w" aria-expanded="true" aria-controls="collapseSix_c_w">
              Treatment
            </button>
          </h5>
        </div>
        <div id="collapseSix_c_w" class="collapse" aria-labelledby="headingSix_c_w" data-parent="#accordion_c_w">
          <div class="card-body">
          </div>
        </div>
      </div>
    
      <div class="card">
        <div class="card-header" id="headingSeven_c_w">
          <h5 class="mb-0">
            <button class="btn btn-link collapsed" data-toggle="collapse" data-target="#collapseSeven_c_w" aria-expanded="true" aria-controls="collapseSeven_c_w">
            Welfare
            </button>
          </h5>
        </div>
        <div id="collapseSeven_c_w" class="collapse" aria-labelledby="headingSeven_c_w" data-parent="#accordion_c_w">
          <div class="card-body">
          </div>
        </div>
      </div>
    

    


    <script src="https://cdnjs.cloudflare.com/ajax/libs/jquery/3.4.1/jquery.js"></script>
    <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/js/bootstrap.min.js"></script>
    <script type="text/javascript">
        $(document).ready(function(){
                $('#getdescription_w').on('click', function(){
                $('#getdescription_w').hide();
                $('#name_title').hide();
                $('#accordion_c_w').show()
                $.getJSON("/raw_recruit_w/{{company_name}}/{{title2}}", function(data_c_w){
                    console.log(data_c_w);
                    var company_name="<p>"+data_c_w.company_name+"</p>"
                        title = "<p>" + data_c_w.title + "</p>";
                        introduction = "<p>" + data_c_w.introduction + "</p>";
                        task = "<p>" + data_c_w.task+"  "+ + "</p>";
                        ability = "<p>" + data_c_w.ability + "</p>";
                        industry = "<p>" + data_c_w.industry + "</p>";
                        registration = "<p>" + data_c_w.registration+"  "+ + "</p>";
                        treatment = "<p>" + data_c_w.treatment + "</p>";
                        welfare = "<p>" + data_c_w.welfare + "</p>";

                    $(".data_c_w").append(company_name);
                    $("#collapseOne_c_w").append(title);
                    $("#collapseTwo_c_w").append(introduction);
                    $("#collapseThree_c_w").append(task); 
                    $("#collapseFour_c_w").append(ability);
                    $("#collapseFive_c_w").append(registration);
                    $("#collapseSix_c_w").append(treatment);
                    $("#collapseSeven_c_w").append(welfare);
                });
              });
            
                $('#getreview').on('click', function(){
                $('#getreview').hide();
                $('#name_title').hide();
                $('#accordion').show()
                $.getJSON("/raw_review/{{company_name}}/{{title}}", function(data){
                    console.log(data);                   
                    var company_name="<p>"+data.company_name+"</p>"
                    var link = "<a href="+data.link +">"+data.link+"</a>";
                    var review_num = "<p>" + data.review_num + "</p>";
                    var stats = "<p>" + JSON.stringify(data.stats) + "</p>";
                        person = "<p>" + data.person+"  "+ + "</p>";
                        title = "<p>" + data.title + "</p>";
                        strength = "<p>" + data.strength + "</p>";
                        weakness = "<p>" + data.weakness + "</p>";
                        want = "<p>" + data.want + "</p>";
                        
                    $(".data").append(company_name);
                    $("#collapseOne").append(link);
                    $("#collapseTwo").append(review_num);
                    $("#collapseThree").append(stats); 
                    $("#collapseFour").append(person);
                    $("#collapseFive").append(title);
                    $("#collapseSix").append(strength);
                    $("#collapseSeven").append(weakness);
                    $("#collapseEight").append(want);
                    
                });
            });     
        });
        $('.collapse').collapse();
        
    </script>
</body>

</html>

Overwriting jobflask/templates/index_w.html
